In [154]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
import time, os
import re

## Downloading and casting to dataframe

In [46]:
with open('videos_info.pkl', 'rb') as picklefile:
    videos_info = pickle.load(picklefile)
with open('channels_about.pkl', 'rb') as picklefile:
    channels_about = pickle.load(picklefile)
with open('channels_videos_page.pkl', 'rb') as picklefile:
    channels_videos_page = pickle.load(picklefile)

In [272]:
channels_about_df = pd.DataFrame.from_dict(channels_about).T
channels_videos_page_df = pd.DataFrame.from_dict(channels_videos_page).T
videos_info_df = pd.DataFrame.from_dict(videos_info).T

## Cleaning channels_videos_page_df

In [321]:
channels_videos_page_df.head(3)

,channel_URL,video_order_num,duration,title,views_video,video_age_min
0,https://www.youtube.com/user/yogawithadriene,1,2100,Yoga for Vulnerability | Move #withme | Yo...,3440914.0,5760.0
1,https://www.youtube.com/channel/UCahoXO8yh6mqJ...,1,2040,30 Min Yoga Flow: Cleanse Your Sacral Chakra |...,25491.0,1440.0
2,https://www.youtube.com/channel/UCLevFC-tyOhyy...,1,761,Hip Mobility Class l Lizette Pompa Yoga,29437.0,302400.0


In [274]:
channels_videos_page_df = channels_videos_page_df.reset_index()

In [275]:
videos_scraped = channels_videos_page_df.columns[1:]

In [276]:
channels_videos_page_df = pd.melt(channels_videos_page_df, id_vars='index', value_vars = videos_scraped)

In [277]:
splitted_value_col = channels_videos_page_df['value'].apply(pd.Series)
channels_videos_page_df = pd.concat([channels_videos_page_df, splitted_value_col], axis= 1)

In [278]:
channels_videos_page_df.columns

Index(['index', 'variable', 'value', 0, 'duration', 'title', 'views_days'], dtype='object')

In [279]:
channels_videos_page_df = channels_videos_page_df[['index', 'variable', 'duration', 'title', 'views_days']]
channels_videos_page_df.rename(columns = {'index':'channel_URL', 'variable':'video_order_num'}, inplace=True)

In [280]:
channels_videos_page_df.shape

(17640, 5)


Cleaning: Duration

In [281]:
channels_videos_page_df['duration'].isna().sum()

2984

NaNs is a result of video tab having no content: https://www.youtube.com/user/yogatoday, https://www.youtube.com/user/yogaworks
Here function didn't work: https://www.youtube.com/user/divinedesignwellness

In [282]:
channels_videos_page_df[channels_videos_page_df['duration'].isna()].sample(6)

,channel_URL,video_order_num,duration,title,views_days
17248,https://www.youtube.com/channel/UC-enqt3oO6foL...,30,NaN,NaN,NaN
16553,https://www.youtube.com/channel/UC7a5LhGLCvslZ...,29,NaN,NaN,NaN
15466,https://www.youtube.com/user/SiyaYoga,27,NaN,NaN,NaN
16633,https://www.youtube.com/channel/UCRocO3B31Q02r...,29,NaN,NaN,NaN
14896,https://www.youtube.com/channel/UC-enqt3oO6foL...,26,NaN,NaN,NaN
8474,https://www.youtube.com/user/divinedesignwellness,15,NaN,NaN,NaN


In [283]:
channels_videos_page_df = channels_videos_page_df[channels_videos_page_df['duration'].notna()]

In [284]:
channels_videos_page_df['duration'] = channels_videos_page_df['duration'].str.replace('- Duration:','').str.replace('.','')

In [285]:
channels_videos_page_df['duration'] = channels_videos_page_df['duration'].str.replace(' minutes', 'M').str.replace(' seconds', 'S').str.replace(' hours', 'H').str.replace(' minute', 'M').str.replace(' second', 'S').str.replace(' hour', 'H').str.replace(',', '').str.replace(' ', '') 

In [286]:
def parse_duration(string):
    try:
        dur = pd.to_datetime(string, format = '%MM%SS') 
    except:
        try:
            dur = pd.to_datetime(string, format = '%HH%MM')
        except:
            try:
                dur = pd.to_datetime(string, format = '%HH%MM%SS')
            except:
                try:
                    dur = pd.to_datetime(string, format = '%HH')
                except:
                    try:
                        dur = pd.to_datetime(string, format = '%MM')
                    except:
                        try:
                            if int(string.replace('S','')) > 60:
                                string = '60S'  #videos with duration a little over 1 min display sec > 60
                            dur = pd.to_datetime(string, format = '%SS')
                        except:
                            dur = ''
    return dur

In [287]:
channels_videos_page_df['duration'] = channels_videos_page_df['duration'].apply(parse_duration)

In [288]:
channels_videos_page_df['duration'] = (channels_videos_page_df['duration'].dt.hour * 60 + channels_videos_page_df['duration'].dt.minute) * 60 + channels_videos_page_df['duration'].dt.second

Cleaning: Views & Days

In [289]:
channels_videos_page_df['views_days'].isna().sum()

0

In [290]:
channels_videos_page_df['views_days'] = channels_videos_page_df['views_days'].str.replace('No', '0')
channels_videos_page_df['views_days'] = channels_videos_page_df['views_days'].str.replace('views', 'view')

In [291]:
channels_videos_page_df['views_video'] = channels_videos_page_df['views_days'].apply(lambda string: (re.sub('[^0-9]+', '', string)))

In [303]:
channels_videos_page_df['views_video']=pd.to_numeric(channels_videos_page_df['views_video'], errors = 'coerce')

In [292]:
channels_videos_page_df['video_age'] = channels_videos_page_df['views_days'].apply(lambda string: re.sub('([0-9]+,[0-9]+ view)|([0-9]+ view)', '', string).replace(' ago', ''))

In [293]:
channels_videos_page_df['video_age'] = channels_videos_page_df['video_age'].apply(lambda string: (re.sub('s', '', string)))
channels_videos_page_df['video_age'] = channels_videos_page_df['video_age'].apply(lambda string: (re.sub(',', '.', string)))

In [294]:
channels_videos_page_df['video_age'].unique()

array(['4 day', '1 day', '7 month', '6 year', '3 day', '14 hour',
       '2 year', '5 year', '12 hour', '2 week', '1 year', '3 week',
       '1 hour', '5 day', '1 week', '4 hour', '9 month', '2 day',
       '2 month', '23 hour', '1 month', '8 month', '3 year', '6 day',
       '32 minute', '11 hour', '3 month', 'Start:', '4 year', '3 hour',
       '1.6 year', '19 hour', '6 hour', '6 month', '22 hour', '9 minute',
       '5 month', '2 hour', '8 hour', '7 hour', '4 week', '18 hour',
       '10 hour', '9 hour', '56 minute', '26 minute', '10 month',
       '16 hour', '7 minute', '4 month', '15 hour', '11 month',
       '28 minute', '20 hour', '5 hour', '21 hour', '17 hour', '1.4 year',
       '3.2 year', '5.1 year', '11.3 year', '13 hour', '39 minute',
       '22.1 year', '8.3 year', '7 year', '26.1 year', '1.2 month',
       '1.3 year', '27.5 year', '6.3 year', '2.1 year', '1.2 year',
       '6.1 year', '15.3 year', '13.3 year', '8 year', '4.6 year',
       '8.4 year', '1.8 year', '3.3 yea

In [295]:
def video_age_to_min(age):
    if 'minute' in age:
        video_age = float(age.replace('minute',''))
    elif 'hour' in age:
        video_age = float(age.replace('hour','')) * 60
    elif 'day' in age:
        video_age = float(age.replace('day','')) * 60 * 24
    elif 'month' in age:
        video_age = float(age.replace('month','')) * 60 * 24 * 30
    elif 'year' in age:
        video_age = float(age.replace('year','')) * 60 * 24 * 30 * 365
    else:
        video_age = 0.0
    return video_age

In [296]:
channels_videos_page_df['video_age_min'] = channels_videos_page_df['video_age'].apply(video_age_to_min)

Miscellaneous

In [309]:
channels_videos_page_df['video_order_num']=pd.to_numeric(channels_videos_page_df['video_order_num'], errors = 'coerce')

In [297]:
channels_videos_page_df['video_age_min'].isna().sum()

0

In [298]:
channels_videos_page_df = channels_videos_page_df[['channel_URL', 
                                                   'video_order_num', 
                                                   'duration', 
                                                   'title', 
                                                   'views_video',
                                                   'video_age_min']]

In [300]:
channels_videos_page_df.shape

(14656, 6)

In [304]:
channels_videos_page_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14656 entries, 0 to 17639
Data columns (total 6 columns):
channel_URL        14656 non-null object
video_order_num    14656 non-null object
duration           14656 non-null int64
title              14656 non-null object
views_video        14611 non-null float64
video_age_min      14656 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 801.5+ KB


In [306]:
#NaNs are premieres, remove those.
channels_videos_page_df[channels_videos_page_df['views_video'].isna()].head()

,channel_URL,video_order_num,duration,title,views_video,video_age_min
78,https://www.youtube.com/channel/UCMOuxZneKXsKi...,1,325,Yoga Tutorial For Beginners - Supine Twist wit...,NaN,0.0
160,https://www.youtube.com/user/yogavedanta,1,4620,"Action, renunciation and indifference to objec...",NaN,0.0
171,https://www.youtube.com/user/AustSchoolMeditation,1,1620,Energising Morning Meditation with Vrindavan d...,NaN,0.0
465,https://www.youtube.com/user/YogaVidyaVortrag,1,411,Shankaracharyas Lehren - Sukadev | Vedanta | M...,NaN,0.0
748,https://www.youtube.com/user/yogavedanta,2,5160,Devotion and dharma - James Swartz - Yoga of L...,NaN,0.0


In [307]:
channels_videos_page_df = channels_videos_page_df[channels_videos_page_df['views_video'].notna()]

In [310]:
channels_videos_page_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14611 entries, 0 to 17639
Data columns (total 6 columns):
channel_URL        14611 non-null object
video_order_num    14611 non-null int64
duration           14611 non-null int64
title              14611 non-null object
views_video        14611 non-null float64
video_age_min      14611 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 799.0+ KB


In [311]:
channels_videos_page_df.head(3)

,channel_URL,video_order_num,duration,title,views_video,video_age_min
0,https://www.youtube.com/user/yogawithadriene,1,2100,Yoga for Vulnerability | Move #withme | Yo...,3440914.0,5760.0
1,https://www.youtube.com/channel/UCahoXO8yh6mqJ...,1,2040,30 Min Yoga Flow: Cleanse Your Sacral Chakra |...,25491.0,1440.0
2,https://www.youtube.com/channel/UCLevFC-tyOhyy...,1,761,Hip Mobility Class l Lizette Pompa Yoga,29437.0,302400.0


## Cleaning channels_about_df

In [346]:
channels_about_df.head(3)

,channel_URL,channel_name,subscriber_count,total_views_channel,channel_created_date,n_featured_channels,n_promo_links,channel_desc_length,channel_desc
0,https://www.youtube.com/user/yogawithadriene,YogaWithAdriene.com,6800000.0,557831762,2012-08-30,3,10,946,WELCOME to Yoga With Adriene! Our mission is t...
1,https://www.youtube.com/user/CorePowerYoga,corepoweryoga.com,17200.0,452439,2008-09-16,0,10,241,Stream a live class or access our library of 2...
2,https://www.youtube.com/channel/UCE92Hzq1rGv3d...,Free Online Resource Library,1180.0,94978,2014-05-27,0,10,420,"Ally Boothroyd is a Kripalu Yoga Teacher, yoga..."


In [313]:
channels_about_df = channels_about_df.reset_index()

In [315]:
channels_about_df.columns

Index(['index', 'name', 'subscriber_count', 'total_views', 'joined_date',
       'n_featured_channels', 'n_links', 'description_length', 'description'],
      dtype='object')

In [322]:
channels_about_df.rename(columns = {'index':'channel_URL', 
                                          'name':'channel_name', 
                                          'total_views': 'total_views_channel', 
                                          'joined_date':'channel_created_date', 
                                          'n_links':'n_promo_links', 
                                          'description_length':'channel_desc_length', 
                                          'description': 'channel_desc'}, inplace=True)

Cleaning Subscriber Count

In [324]:
channels_about_df['subscriber_count'].isna().sum()

0

In [325]:
def strval_to_float(subscriber_count):
    if type(subscriber_count) == float or type(subscriber_count) == int:
        sc = subscriber_count
    elif 'K' in subscriber_count:
        sc = float(subscriber_count.replace('K',''))*1000
    elif 'M' in subscriber_count:
        sc = float(subscriber_count.replace('M',''))*1000000
    else:
        sc = 0.0
    return sc

In [326]:
channels_about_df['subscriber_count'] = channels_about_df['subscriber_count'].apply(lambda string: (strval_to_float(string)))

Cleaning Total Views by Channel

In [333]:
channels_about_df['total_views_channel'].isna().sum()

0

In [331]:
channels_about_df['total_views_channel'] = channels_about_df['total_views_channel'].str.replace(',', '')

In [332]:
channels_about_df['total_views_channel'] = pd.to_numeric(channels_about_df['total_views_channel'], errors = 'coerce')

Cleaning Channel Created Date

In [341]:
channels_about_df['channel_created_date'].isna().sum()

0

In [336]:
channels_about_df['channel_created_date'] = channels_about_df['channel_created_date'].str.replace('Joined ', '')

In [340]:
channels_about_df['channel_created_date'] = pd.to_datetime(channels_about_df['channel_created_date'])

Miscellaneous

In [345]:
channels_about_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 9 columns):
channel_URL             367 non-null object
channel_name            367 non-null object
subscriber_count        367 non-null float64
total_views_channel     367 non-null int64
channel_created_date    367 non-null datetime64[ns]
n_featured_channels     367 non-null int64
n_promo_links           367 non-null int64
channel_desc_length     367 non-null int64
channel_desc            367 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 25.9+ KB


In [344]:
channels_about_df['channel_desc_length']=pd.to_numeric(channels_about_df['channel_desc_length'], errors = 'coerce')
channels_about_df['n_promo_links']=pd.to_numeric(channels_about_df['n_promo_links'], errors = 'coerce')
channels_about_df['n_featured_channels']=pd.to_numeric(channels_about_df['n_featured_channels'], errors = 'coerce')

In [347]:
channels_about_df.head(3)

,channel_URL,channel_name,subscriber_count,total_views_channel,channel_created_date,n_featured_channels,n_promo_links,channel_desc_length,channel_desc
0,https://www.youtube.com/user/yogawithadriene,YogaWithAdriene.com,6800000.0,557831762,2012-08-30,3,10,946,WELCOME to Yoga With Adriene! Our mission is t...
1,https://www.youtube.com/user/CorePowerYoga,corepoweryoga.com,17200.0,452439,2008-09-16,0,10,241,Stream a live class or access our library of 2...
2,https://www.youtube.com/channel/UCE92Hzq1rGv3d...,Free Online Resource Library,1180.0,94978,2014-05-27,0,10,420,"Ally Boothroyd is a Kripalu Yoga Teacher, yoga..."


## Cleaning videos_info_df

In [400]:
videos_info_df = pd.DataFrame.from_dict(videos_info).T

In [401]:
videos_info_df.tail(3)

Title  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  Ben & Jimmy's Guide To London: First Impressions   
                                      8hHj27QazIQ     Ben & Jimmy's Guide To London: Londoners E1/3   
                                      3RifUHkAon0                Tyra Banks Makeup - Models' Corner   

                                                  Views  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  2015   
                                      8hHj27QazIQ  1534   
                                      3RifUHkAon0  3128   

                                                                                         Description  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  In honour of YouTube Comedy Week we've launche...   
                                      8hHj27QazIQ  This week is YouTube Comedy Week, so we are br...   
                                      3RifUHkAon0  Tyra Banks always has defined cheekbones and s...   

                                                            Publication Date  \
https://www.youtube.com/user/VideoJug A0GC4z60TPE  Published on May 22, 2013   
                                      8hHj27QazIQ  Published on May 20, 2013   
                                      3RifUHkAon0  Published on May 16, 2013   

                                                  Likes Dislikes  
https://www.youtube.com/user/VideoJug A0GC4z60TPE    20        4  
                                      8hHj27QazIQ    19        4  
                                      3RifUHkAon0    52        4

In [402]:
videos_info_df = videos_info_df.reset_index()

In [403]:
videos_info_df.columns

Index(['level_0', 'level_1', 'Title', 'Views', 'Description',
       'Publication Date', 'Likes', 'Dislikes'],
      dtype='object')

In [404]:
videos_info_df = videos_info_df[['level_0', 'level_1', 'Title', 'Views', 'Description', 'Publication Date', 'Likes', 'Dislikes']]
videos_info_df.rename(columns = {'level_0':'channel_URL', 
                                 'level_1':'video_URL_ext', 
                                 'Title':'title', 
                                 'Views':'views_DUPL', 
                                 'Description':'video_desc', 
                                 'Likes':'likes', 
                                 'Dislikes':'dislikes', 'Publication Date': 'video_published_date'}, inplace=True)

In [405]:
videos_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9399 entries, 0 to 9398
Data columns (total 8 columns):
channel_URL             9399 non-null object
video_URL_ext           9399 non-null object
title                   9399 non-null object
views_DUPL              9399 non-null object
video_desc              9399 non-null object
video_published_date    9399 non-null object
likes                   9399 non-null object
dislikes                9399 non-null object
dtypes: object(8)
memory usage: 587.6+ KB


In [406]:
videos_info_df['video_desc_length'] = videos_info_df['video_desc'].apply(len)

Cleaning Video Published Date

In [421]:
videos_info_df['video_published_date'] = videos_info_df['video_published_date'].str.replace('Streamed live .+ ago', 'Streamed live on Apr 9, 2020')

In [415]:
pattern = re.compile("Published|Premiered|Streamed live")
#pattern1 = re.compile(r'Premiered')
videos_info_df['delivery method'] = videos_info_df['video_published_date'].apply(lambda string: str(pattern.findall(string)))

In [416]:
#891 live, not published
videos_info_df['delivery method'].unique()

array(["['Published']", "['Streamed live']", "['Premiered']"],
      dtype=object)

In [417]:
videos_info_df[videos_info_df['is_published'] == '[]'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 10 columns):
channel_URL             0 non-null object
video_URL_ext           0 non-null object
title                   0 non-null object
views_DUPL              0 non-null object
video_desc              0 non-null object
video_published_date    0 non-null object
likes                   0 non-null object
dislikes                0 non-null object
video_desc_length       0 non-null int64
is_published            0 non-null object
dtypes: int64(1), object(9)
memory usage: 0.0+ bytes


In [422]:
videos_info_df[videos_info_df['is_published'] == "['Streamed live']"].sample(20)

,channel_URL,video_URL_ext,title,views_DUPL,video_desc,video_published_date,likes,dislikes,video_desc_length,is_published
8906,https://www.youtube.com/channel/UCEl4J6Zg3tlPu...,Q9mQEIviT7Y,Sivananda Yoga Live class - 29 March 8am,834,"@Sivananda Yoga Centre, Gurgaon Continuing an ...","Streamed live on Mar 28, 2020",13,0,792,['Streamed live']
5906,https://www.youtube.com/user/DrMelissaWest,sy9U_8-wg40,Winter Solstice Restorative Yoga | Yoga with M...,1971,Winter solstice occurs between December 20 and...,"Streamed live on Dec 20, 2019",98,0,1095,['Streamed live']
1509,https://www.youtube.com/channel/UC7a5LhGLCvslZ...,VXQZLtBPCoI,Online Yoga for the Quarantined - Day 4 - Inte...,187,Join me again as I provide you with daily yoga...,"Streamed live on Mar 23, 2020",10,0,255,['Streamed live']
2926,https://www.youtube.com/user/jockestonia,QnogKToi4ZE,Ashtanga Monkey L II day 2 continues,455,http://www.ashtangamonkey.com,"Streamed live on Sep 30, 2017",3,0,29,['Streamed live']
7014,https://www.youtube.com/channel/UC0n6WQ3_PAePX...,syFpUmwZGSM,Day ONE of LIVE Jesus-Centered Yoga!,8338,Welcome to Day ONE of Yoga Week – FOUR DAYS of...,"Streamed live on Feb 24, 2020",298,7,1227,['Streamed live']
8519,https://www.youtube.com/user/yogatic,AFnIYDylLJo,LIVE 31 March 2020 Silver lining - Finding res...,4059,Come join me and our EkhartYoga community from...,"Streamed live on Mar 31, 2020",171,3,1984,['Streamed live']
8518,https://www.youtube.com/user/yogatic,JJMCnOuhtSk,LIVE 1 April 2020 Taking refuge - Finding resi...,3785,Come join me and our EkhartYoga community from...,"Streamed live on Apr 1, 2020",187,0,1984,['Streamed live']
2687,https://www.youtube.com/channel/UCJP1ionI2lIe6...,8gVrffMitII,Healthy Backs Level 1.5,26,"Join me for a rejuvenating mix of strength, ba...","Streamed live on Mar 23, 2020",1,0,126,['Streamed live']
594,https://www.youtube.com/user/yogilou8,l9LaMXA5rnA,SucramYoga for fitness Level 1 & 2,87,,"Streamed live on Mar 31, 2020",5,0,0,['Streamed live']
3203,https://www.youtube.com/channel/UC-enqt3oO6foL...,dygp9XC7Tb8,Yoga: les secrets de la longévité.,883,"Aujourd'hui, à travers une série de postures s...","Streamed live on Nov 17, 2019",28,1,204,['Streamed live']


In [ ]:
pattern = re.compile('([0-9]+,[0-9]+ views)|([0-9]+ views)')
video_channel_df_nona['views_video'] = video_channel_df_nona['views_days'].apply(lambda string: str(pattern.findall(string)))#.str.replace('/'',''))

In [362]:
videos_info_df['video_published_date'].isna().sum()

0

In [365]:
videos_info_df['video_published_date'] = videos_info_df['video_published_date'].str.replace('Published on ', '')

In [372]:
videos_info_df['video_published_date'] = videos_info_df['video_published_date'].str.replace('Streamed live on ', '')

In [375]:
videos_info_df['video_published_date'] = videos_info_df['video_published_date'].str.replace('Streamed live ', '').str.replace(' ago', '')


In [377]:
videos_info_df['video_published_date'].unique()

array(['Apr 4, 2020', 'Feb 29, 2020', 'Feb 1, 2020', ..., 'May 24, 2013',
       'May 22, 2013', 'May 16, 2013'], dtype=object)

In [380]:
videos_info_df['video_published_date'].sample(20)

5812     Feb 9, 2017
7830    Jan 30, 2020
4046     Nov 2, 2019
8802     Apr 3, 2020
698      Oct 3, 2019
4113    Mar 28, 2020
5471    Oct 28, 2019
7777    Dec 21, 2019
8760     Feb 5, 2015
6415    Mar 31, 2019
7920    Feb 21, 2020
7461     Mar 9, 2020
5359     May 2, 2018
309     Mar 29, 2020
5056     Mar 3, 2019
5603     Jan 1, 2020
471      Nov 9, 2018
6837    Apr 17, 2015
4111    Jan 24, 2019
4148    Mar 23, 2020
Name: video_published_date, dtype: object

In [376]:
videos_info_df['video_published_date'] = pd.to_datetime(videos_info_df['video_published_date'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-01 05:00:00